<a href="https://colab.research.google.com/github/davidsonsantana89/dass--CESAR_SCHOOL-Topicos_Contemporaneos/blob/main/exercicios/parte_2/dass_10_Agents.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

DAVIDSON ALVES SANTOS DE SANTANA

dass@cesar.school

In [1]:
import getpass
import os

os.environ["OPENAI_API_KEY"] = getpass.getpass()

··········


In [2]:
!pip install langchain_openai langchain_core langchain_experimental langchain.tools

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.0/52.0 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 396.2/396.2 kB 21.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 204.3/204.3 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 70.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 43.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.7/150.7 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.7/365.7 kB 21.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 43.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/

In [3]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

In [7]:
llm.invoke("Qual o tempo em Recife?")

AIMessage(content='Desculpe, mas não consigo fornecer informações em tempo real, como a previsão do tempo. Recomendo que você verifique um site de meteorologia ou um aplicativo de clima para obter as informações mais atualizadas sobre o tempo em Recife.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 48, 'prompt_tokens': 13, 'total_tokens': 61}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_f33667828e', 'finish_reason': 'stop', 'logprobs': None}, id='run-9bdf276f-1c7d-4339-8ce0-bf61e6a2474d-0', usage_metadata={'input_tokens': 13, 'output_tokens': 48, 'total_tokens': 61})

In [22]:
import random
from langchain.tools import BaseTool, StructuredTool, tool

@tool
def weather(query: str) -> str:
    """Procura pelo clima de uma cidade online"""
    weather = ["ensolarado", "nublado", "chuvoso", "neve"]
    result = random.choice(weather)
    return result

@tool
def multiply(a: int, b: int) -> int:
    """Multiplica dois números"""
    return a * b

@tool
def write_file(file_name: str, content: str) -> None:
    """Escreve um arquivo de texto"""
    with open(file_name, "w") as f:
        f.write(content)

@tool
def count_letters(word: str, letter: str) -> int:
    """Conta a quantidade de ocorrências de uma letra aparece em uma palavra"""
    count = 0
    for char in word:
        if char == letter:
            count += 1
    return count

tools = [weather, multiply, write_file, count_letters]
llm_with_tools = llm.bind_tools(tools)

In [23]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Você é um assistente mas não sabe do clima ou fazer cálculos matemáticos.",
        ),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

In [24]:
from langchain.agents.format_scratchpad.openai_tools import format_to_openai_tool_messages
from langchain.agents.output_parsers.openai_tools import OpenAIToolsAgentOutputParser
from langchain.agents import AgentExecutor

agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_to_openai_tool_messages(
            x["intermediate_steps"]
        ),
    }
    | prompt
    | llm_with_tools
    | OpenAIToolsAgentOutputParser()
)

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [19]:
agent_executor.invoke({"input": "Qual o clima em Recife?"})



> Entering new AgentExecutor chain...

Invoking: `weather` with `{'query': 'Recife'}`


neveO clima em Recife está com neve. Isso é bastante incomum para a região, que geralmente tem um clima tropical. Se precisar de mais informações ou detalhes, é só avisar!

> Finished chain.


{'input': 'Qual o clima em Recife?',
 'output': 'O clima em Recife está com neve. Isso é bastante incomum para a região, que geralmente tem um clima tropical. Se precisar de mais informações ou detalhes, é só avisar!'}

## Exercícios

### Exercício 1
Adicione uma função nas ferramentas do agente para criar arquivos de texto. Em seguida, teste o agente.

In [20]:
agent_executor.invoke({"input": "Crie um arquivo de texto contendo informações sobre o clima de Recife."})



> Entering new AgentExecutor chain...

Invoking: `weather` with `{'query': 'Recife'}`


neve
Invoking: `write_file` with `{'file_name': 'clima_recife.txt', 'content': 'Informações sobre o clima de Recife:\n\n- Condição: Neve\n'}`


NoneO arquivo de texto contendo informações sobre o clima de Recife foi criado com sucesso. Ele contém a seguinte informação:

```
Informações sobre o clima de Recife:

- Condição: Neve
```

> Finished chain.


{'input': 'Crie um arquivo de texto contendo informações sobre o clima de Recife.',
 'output': 'O arquivo de texto contendo informações sobre o clima de Recife foi criado com sucesso. Ele contém a seguinte informação:\n\n```\nInformações sobre o clima de Recife:\n\n- Condição: Neve\n```'}

### Exercício 2
Crie um agente que conte a quantidade de ocorrências de uma determinada letra em uma palavra. Em seguida, teste o agente.

In [25]:
agent_executor.invoke({"input": "quantas letras R existem em strawberry?"})



> Entering new AgentExecutor chain...

Invoking: `count_letters` with `{'word': 'strawberry', 'letter': 'r'}`


3A palavra "strawberry" contém 3 letras "R".

> Finished chain.


{'input': 'quantas letras R existem em strawberry?',
 'output': 'A palavra "strawberry" contém 3 letras "R".'}